In [7]:
%load_ext autoreload
from scipy import sparse
import numpy as np
import cplex as cp
from cplex.exceptions import CplexError
from importlib import reload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


cpx.f     = obj.c;
cpx.sense = 'minimize';
cpx.Aineq = obj.A_ineq;
cpx.bineq = obj.b_ineq;
cpx.Aeq   = obj.A_eq;
cpx.beq   = obj.b_eq;
cpx.lb    = obj.lb;
cpx.ub    = obj.ub;
cpx.ctype = repmat('C',1,size(obj.A_ineq,2));
cpx.ctype(obj.idx_z) = 'B';
obj.indicators.z',...
obj.indicators.inverse',...
obj.indicators.A',1,...
obj.indicators.sense',... 
obj.indicators.b',...
'',...
obj.indicators.type',...
obj.indicators.epln');

cplexmiqp(H, f, Aineq, bineq, Aeq, beq, l, Q, r, sostype, sosind, soswt, lb, ub, ctype, x0, options)

In [8]:
# example setup (default: minimize)
# main problem
c = [-1,0,0,0]
A_ineq = sparse.csr_matrix([[1,1,0,0],[-1,0,0,0]])
b_ineq = [3,-1]
A_eq = sparse.csr_matrix([[0,1,0,0]])
b_eq = [0.6]
lb = [1,0,0,0]
ub = [np.inf, np.inf, 1, 1]
vtype = 'CCBB'
# indicator constraints
ic_binv = [2,3]
ic_A = sparse.csr_matrix([[1,1,0,0],[1,0,0,0]])
ic_b = [0,1.2]
ic_sense = "LE"
ic_indicval = [1,0]
indic_constr = mcs.Indicator_constraints(ic_binv, ic_A, ic_b, ic_sense, ic_indicval)

In [11]:
%autoreload
my_prob = mcs.MILP_LP(  c=c,
                        A_ineq=A_ineq,
                        b_ineq=b_ineq,
                        A_eq=A_eq,
                        b_eq=b_eq,
                        lb=lb,
                        ub=ub,
                        vtype=vtype, 
                        indic_constr=indic_constr,
                        solver='cplex')
x, opt_cx, status = my_prob.solve()
# solution.get_status() returns an integer code
print("Solution status = ", str(status))
# the following line prints the corresponding string
print("Solution value  = ", str(round(opt_cx,5)))
sol = {'x'+str(i) : round(x[i],5) for i in range(len(x))}
print(sol)

Solution status =  0
Solution value  =  -2.4
{'x0': 2.4, 'x1': 0.6, 'x2': 0, 'x3': 1}


In [1]:
import ray
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': 'tcp://127.0.0.1:57667',
 'raylet_socket_name': 'tcp://127.0.0.1:53470',
 'webui_url': None,
 'session_dir': 'C:\\Users\\phili\\AppData\\Local\\Temp\\ray\\session_2022-01-27_15-18-15_214806_30504',
 'metrics_export_port': 61220,
 'node_id': '904cb37b6cf3c81297f5f258aa4f0f8c2f84e21ee93e117ff1b0217e'}

In [2]:
%load_ext autoreload
import cobra
from importlib import reload
ecc2 = cobra.io.read_sbml_model("ECC2comp.sbml")
iml1515 = cobra.io.read_sbml_model("iML1515.sbml")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-09-03


In [18]:
%autoreload
import mcs
from mcs import fva
import cProfile
reload(mcs)
# opt = ecc2.optimize()
# print(opt)
# opt2 = fba(ecc2)
# print(opt2)
# opt3 = cobra.flux_analysis.variability.flux_variability_analysis(ecc2)
# print(opt3)
# cProfile.runctx("mcs.fva(iml1515)",globals(),locals())
opt4 = mcs.fva(iml1515)
print(opt4)
# cProfile.runctx()

         minimum  maximum
CYTDK2       0.0     -0.0
XPPT         0.0     -0.0
HXPRT        0.0     -0.0
NDPK5        0.0     -0.0
SHK3Dr       0.0     -0.0
...          ...      ...
MPTS         0.0     -0.0
MOCOS        0.0     -0.0
BMOGDS2      0.0     -0.0
FESD2s       0.0     -0.0
OCTNLL       0.0     -0.0

[2712 rows x 2 columns]
initialized
         minimum  maximum
CYTDK2       0.0     -0.0
XPPT         0.0     -0.0
HXPRT        0.0     -0.0
NDPK5        0.0     -0.0
SHK3Dr       0.0     -0.0
...          ...      ...
MPTS         0.0     -0.0
MOCOS        0.0     -0.0
BMOGDS2      0.0     -0.0
FESD2s       0.0     -0.0
OCTNLL       0.0     -0.0

[2712 rows x 2 columns]


In [1]:
ray.shutdown()

NameError: name 'ray' is not defined

In [9]:
fv,max_v = cplex_interface.cplex_fba(ecc2)

In [10]:
lb,ub = cplex_interface.cplex_fva(ecc2)

In [78]:
# Function Decorator 1
def show_steps(fcn):
    def addition(a,b):
        for i in range(1,b+1):
            print(fcn(i,a))
    return addition

@show_steps
def mtpl(a,b):
    return a*b

#mtpl = show_steps(mtpl)
mtpl(5,3)

5
10
15


TypeError: addition() missing 1 required positional argument: 'b'

In [100]:
# Function Decorator 2
def show_and_add(c):
    def show_steps(fcn):
        def addition(a,b):
            for i in range(1,b+1):
                print(fcn(i,a)+c)
        return addition
    def show_steps2(fcn1):
        def addition(a,b):
            for i in range(1,a+1):
                print(fcn1(i,b)+c)
        return addition
    if True:
        return show_steps
    else:
        return show_steps2
    

@show_and_add(5)
def mtpl(a,b):
    return a*b

mtpl(4,3)

9
13
17


In [113]:
print([[rid[i],fv[i]] for i in range(0,121)])
print([[rid[i],lb[i],ub[i]] for i in range(0,121)])

[['EX_Biomass', 2.3749950493908365], ['EX_4CRSOL_ex', 0.0005296238960141565], ['EX_5DRIB_ex', 0.0015936216781412514], ['EX_ac_ex', -10.0], ['EX_adp_c', 5.329070518200751e-15], ['EX_AMOB_ex', 4.749990098781673e-06], ['EX_ca2_ex', -0.012361849232079305], ['EX_cl_ex', -0.012361849232079305], ['EX_co2_ex', 52.505721724031396], ['EX_coa_c', 9.499980197493743e-05], ['EX_cobalt2_ex', -5.9374876234770917e-05], ['EX_cu2_ex', -0.0016838714900181031], ['EX_etoh_ex', -2.2737367544323206e-13], ['EX_fe2_ex', -0.019600834142622574], ['EX_fe3_ex', -0.01854396134564365], ['EX_for_ex', 0.0], ['EX_glc_DASH_D_ex', -10.0], ['EX_glyc_ex', -10.0], ['EX_h_ex', -8.159688025504863], ['EX_h2_ex', 0.0], ['EX_h2o_ex', 115.21842075120652], ['EX_k_ex', -0.46357528369059736], ['EX_lac_ex', 0.0], ['EX_meoh_ex', 4.749990098781673e-06], ['EX_mg2_ex', -0.020603082053465508], ['EX_mn2_ex', -0.001641121579129068], ['EX_mobd_ex', -0.00030637436137141786], ['EX_mqn8_c', 0.0], ['EX_MTHTHF_ex', 0.0010639977821270948], ['EX_nad

In [80]:
rid = [i.id for i in ecc2.reactions]

In [51]:
g = [i.objective_coefficient for i in ecc2.reactions]
print(g)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
ecc2.objective_direction

'max'